In [1]:
!pip3 install -qU langgraph
!pip3 install langchain-mcp-adapters
import os
import requests
import json
from openai import OpenAI
import httpx
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.agents import tool
from langchain.agents import initialize_agent, AgentType, load_tools
from langchain_core.tools import Tool
from langchain.tools import tool
from datetime import date
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
import nest_asyncio
import ssl


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
# Set environment variables to disable SSL verification
os.environ['PYTHONHTTPSVERIFY'] = '0'
os.environ['CURL_CA_BUNDLE'] = ''

# Create a custom SSL context that doesn't verify certificates
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

# Override the default SSL context
ssl._create_default_https_context = lambda: ssl_context

In [3]:
nest_asyncio.apply()

test_client = httpx.Client(verify=False, timeout=30.0)

vcapservices = os.getenv('VCAP_SERVICES')
services = json.loads(vcapservices)

def is_chatservice(service):
    return service["name"] == "gen-ai-qwen3-ultra"

chat_services = filter(is_chatservice, services["genai"])
chat_credentials = list(chat_services)[0]["credentials"]

llm = ChatOpenAI(temperature=0.9, model=chat_credentials["model_name"], base_url=chat_credentials["api_base"], api_key=chat_credentials["api_key"], http_client=test_client)

def is_userprovided(service):
    return service["name"] == "time-mcp-server"
user_provided_services = filter(is_userprovided, services["user-provided"])
user_provided_credentials = list(user_provided_services)[0]["credentials"]

print(user_provided_credentials["mcpServiceURL"])
time_mcp = user_provided_credentials["mcpServiceURL"] + "/sse"

def is_petclinic_userprovided(service):
    return service["name"] == "petclinic-mcp-server"
user_provided_petclinic_services = filter(is_petclinic_userprovided, services["user-provided"])
user_provided_petclinic_credentials = list(user_provided_petclinic_services)[0]["credentials"]

print(user_provided_petclinic_credentials["mcpServiceURL"])
petclinic_mcp = user_provided_petclinic_credentials["mcpServiceURL"] + "/sse"


#time tool implemented as langchain tool
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

# Async HTTP client for MCP
#async_client = httpx.AsyncClient(http2=True, verify=False, timeout=10.0)
# Patch httpx globally for MCP client
original_async_client = httpx.AsyncClient

class NoSSLAsyncClient(httpx.AsyncClient):
    def __init__(self, **kwargs):
         kwargs['verify'] = False
         super().__init__(**kwargs)

httpx.AsyncClient = NoSSLAsyncClient

# Create MCP client
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "sse",
            "url": time_mcp
        },
        "petclinic": {
            "transport": "sse",
            "url": petclinic_mcp
        }
    }
)
#Fetch and print tools
tools = await client.get_tools()
print("MCP Tools:", tools)

https://time-mcp-server.apps.tas-ndc.kuhn-labs.com
https://petclinic-mcp.apps.tas-ndc.kuhn-labs.com
MCP Tools: [StructuredTool(name='getCurrentTime', description='Get the current time', args_schema={'type': 'object', 'properties': {}, 'required': [], 'additionalProperties': False}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7f2c79c65080>), StructuredTool(name='fetch_petsByStatus', description='Retrieve pets from petstore api', args_schema={'properties': {'status': {'title': 'Status', 'type': 'string'}}, 'required': ['status'], 'title': 'fetch_petsByStatusArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x7f2c79c66840>)]
